# Parameter Sweeps and Batch Processing

This notebook demonstrates how to use PyAERMOD for parameter studies and batch processing.

## Topics Covered

1. Stack height optimization (parametric study)
2. Emission rate sensitivity analysis
3. Meteorological year comparison
4. Batch processing with AERMODRunner
5. Results analysis and visualization

**Prerequisites:**
- PyAERMOD installed
- AERMOD executable available
- Meteorological data files
- Understanding of batch processing concepts

## Setup

In [ ]:
from pyaermod.input_generator import (
    AERMODProject,
    ControlPathway,
    SourcePathway,
    PointSource,
    ReceptorPathway,
    PolarGrid,
    CartesianGrid,
    MeteorologyPathway,
    OutputPathway,
    PollutantType,
    TerrainType
)

from pyaermod.runner import AERMODRunner
from pyaermod.output_parser import AERMODOutputParser

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

print("✓ PyAERMOD modules imported")

## Example 1: Stack Height Optimization

Systematically test different stack heights to find the minimum height for compliance.

In [ ]:
# Define parameter sweep
stack_heights = [20, 30, 40, 50, 60, 70, 80]  # meters
input_files = []

print("Creating stack height optimization scenarios...")
print("="*70)

for height in stack_heights:
    # Create control pathway
    control = ControlPathway(
        title_one=f"Stack Height: {height}m",
        title_two="Optimization study",
        pollutant_id=PollutantType.SO2,
        averaging_periods=["ANNUAL", "24", "3", "1"],
        terrain_type=TerrainType.FLAT
    )
    
    # Create source with variable height
    sources = SourcePathway()
    sources.add_source(PointSource(
        source_id="STACK1",
        x_coord=0.0,
        y_coord=0.0,
        base_elevation=100.0,
        stack_height=float(height),
        stack_temp=420.0,
        exit_velocity=15.0,
        stack_diameter=2.0,
        emission_rate=5.0  # g/s
    ))
    
    # Dense receptor grid
    receptors = ReceptorPathway()
    receptors.add_polar_grid(PolarGrid(
        x_origin=0.0,
        y_origin=0.0,
        dist_init=100.0, dist_num=10, dist_delta=100.0, dir_init=0, dir_num=36, dir_delta=10
    ))
    
    # Meteorology
    meteorology = MeteorologyPathway(
        surface_file="met.sfc",
        profile_file="met.pfl"
    )
    
    # Output
    output = OutputPathway(
        receptor_table=True,
        max_table=True
    )
    
    # Create project and write
    project = AERMODProject(
        control=control,
        sources=sources,
        receptors=receptors,
        meteorology=meteorology,
        output=output
    )
    
    filename = f"height_sweep_h{height:02d}m.inp"
    project.write(filename)
    input_files.append(filename)
    print(f"✓ Created: {filename}")

print(f"\n✓ Created {len(input_files)} input files for stack height sweep")

### Run Batch Processing

Use `AERMODRunner` to execute all scenarios in parallel.

In [ ]:
# Initialize runner
runner = AERMODRunner(aermod_executable="aermod")

print("Running AERMOD batch processing...")
print("This may take several minutes...\n")

# Run all scenarios with 4 parallel workers
results = runner.run_batch(
    input_files=input_files,
    num_workers=4,
    timeout=300  # 5 minutes per run
)

# Check results
successful = sum(1 for r in results if r['success'])
print(f"\n✓ Completed: {successful}/{len(results)} runs successful")

for result in results:
    status = "✓" if result['success'] else "✗"
    print(f"{status} {result['input_file']}: {result['return_code']}")

### Parse Results and Find Maximum Concentrations

In [ ]:
# Parse all output files
parser = AERMODOutputParser()
max_concentrations = []

for height, inp_file in zip(stack_heights, input_files):
    out_file = inp_file.replace('.inp', '.out')
    
    try:
        results = parser.parse_output_file(out_file)
        
        # Extract maximum concentration from ANNUAL results
        if results['concentrations'] and len(results['concentrations']) > 0:
            conc_df = results['concentrations'][0]  # First table (ANNUAL)
            max_conc = conc_df['CONC'].max()
            max_concentrations.append({
                'height': height,
                'max_conc': max_conc
            })
            print(f"Height {height:2d}m: Max = {max_conc:.2f} μg/m³")
    except Exception as e:
        print(f"Error parsing {out_file}: {e}")

# Create results dataframe
df_results = pd.DataFrame(max_concentrations)
print("\n" + "="*50)
print(df_results.to_string(index=False))

### Visualize Results

In [ ]:
# Plot stack height vs. maximum concentration
plt.figure(figsize=(10, 6))
plt.plot(df_results['height'], df_results['max_conc'], 
         marker='o', linewidth=2, markersize=8, color='steelblue')

# Add standard line (example: NAAQS)
naaqs_annual = 0.03  # Example standard (μg/m³)
plt.axhline(y=naaqs_annual, color='red', linestyle='--', 
            linewidth=2, label=f'Standard ({naaqs_annual} μg/m³)')

plt.xlabel('Stack Height (m)', fontsize=12)
plt.ylabel('Maximum Annual Concentration (μg/m³)', fontsize=12)
plt.title('Stack Height Optimization Study', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)
plt.tight_layout()
plt.savefig('stack_height_optimization.png', dpi=300)
print("✓ Saved: stack_height_optimization.png")
plt.show()

# Find minimum compliant height
compliant = df_results[df_results['max_conc'] <= naaqs_annual]
if not compliant.empty:
    min_height = compliant['height'].min()
    print(f"\n✓ Minimum compliant stack height: {min_height} m")
else:
    print("\n✗ No stack height achieves compliance with tested range")

## Example 2: Emission Rate Sensitivity

Test how concentration scales with emission rate (should be linear).

In [ ]:
# Define emission rates to test
emission_rates = [1.0, 2.0, 5.0, 10.0, 20.0]  # g/s
input_files_emission = []

print("Creating emission rate sensitivity scenarios...")
print("="*70)

for rate in emission_rates:
    control = ControlPathway(
        title_one=f"Emission Rate: {rate} g/s",
        title_two="Sensitivity analysis",
        pollutant_id=PollutantType.PM25,
        averaging_periods=["ANNUAL"],
        terrain_type=TerrainType.FLAT
    )
    
    sources = SourcePathway()
    sources.add_source(PointSource(
        source_id="STACK1",
        x_coord=0.0,
        y_coord=0.0,
        base_elevation=100.0,
        stack_height=50.0,  # Fixed height
        stack_temp=400.0,
        exit_velocity=15.0,
        stack_diameter=2.0,
        emission_rate=rate  # Variable emission rate
    ))
    
    receptors = ReceptorPathway()
    receptors.add_cartesian_grid(
        CartesianGrid.from_bounds(
            x_min=-2000,
            x_max=2000,
            y_min=-2000,
            y_max=2000,
            spacing=100
        )
    )
    
    meteorology = MeteorologyPathway(
        surface_file="met.sfc",
        profile_file="met.pfl"
    )
    
    output = OutputPathway(
        receptor_table=True,
        max_table=True
    )
    
    project = AERMODProject(
        control=control,
        sources=sources,
        receptors=receptors,
        meteorology=meteorology,
        output=output
    )
    
    filename = f"emission_sweep_q{rate:.1f}gs.inp"
    project.write(filename)
    input_files_emission.append(filename)
    print(f"✓ Created: {filename}")

print(f"\n✓ Created {len(input_files_emission)} files for emission rate sweep")
print("\nExpected: Concentration should scale linearly with emission rate")
print("(This validates model setup and can be used for quick scaling)")

## Example 3: Multi-Year Meteorology Comparison

Compare results across different meteorological years.

In [ ]:
# Define meteorological years
met_years = [
    {"year": "2018", "sfc": "met_2018.sfc", "pfl": "met_2018.pfl"},
    {"year": "2019", "sfc": "met_2019.sfc", "pfl": "met_2019.pfl"},
    {"year": "2020", "sfc": "met_2020.sfc", "pfl": "met_2020.pfl"},
]

input_files_met = []

print("Creating multi-year meteorology scenarios...")
print("="*70)

for met in met_years:
    control = ControlPathway(
        title_one=f"Meteorology Year: {met['year']}",
        title_two="Multi-year comparison",
        pollutant_id=PollutantType.NOX,
        averaging_periods=["ANNUAL"],
        terrain_type=TerrainType.FLAT
    )
    
    sources = SourcePathway()
    sources.add_source(PointSource(
        source_id="STACK1",
        x_coord=0.0,
        y_coord=0.0,
        base_elevation=100.0,
        stack_height=45.0,
        stack_temp=400.0,
        exit_velocity=12.0,
        stack_diameter=2.0,
        emission_rate=3.0
    ))
    
    receptors = ReceptorPathway()
    receptors.add_cartesian_grid(
        CartesianGrid.from_bounds(
            x_min=-1500,
            x_max=1500,
            y_min=-1500,
            y_max=1500,
            spacing=75
        )
    )
    
    # Different met files for each year
    meteorology = MeteorologyPathway(
        surface_file=met['sfc'],
        profile_file=met['pfl']
    )
    
    output = OutputPathway(
        receptor_table=True,
        max_table=True
    )
    
    project = AERMODProject(
        control=control,
        sources=sources,
        receptors=receptors,
        meteorology=meteorology,
        output=output
    )
    
    filename = f"met_comparison_{met['year']}.inp"
    project.write(filename)
    input_files_met.append(filename)
    print(f"✓ Created: {filename}")

print(f"\n✓ Created {len(input_files_met)} files for meteorology comparison")
print("\nUse case: Assess year-to-year variability in air quality impacts")

## Example 4: Advanced Batch Processing with Error Handling

In [ ]:
# Example of robust batch processing with error handling
def run_parameter_sweep_robust(input_files, max_workers=4, timeout=300):
    """
    Run batch processing with comprehensive error handling and reporting.
    """
    runner = AERMODRunner(aermod_executable="aermod")
    
    print(f"Starting batch run: {len(input_files)} files")
    print(f"Workers: {max_workers}, Timeout: {timeout}s\n")
    
    results = runner.run_batch(
        input_files=input_files,
        num_workers=max_workers,
        timeout=timeout
    )
    
    # Analyze results
    successful = [r for r in results if r['success']]
    failed = [r for r in results if not r['success']]
    
    print("="*70)
    print("BATCH RUN SUMMARY")
    print("="*70)
    print(f"Total runs:       {len(results)}")
    print(f"Successful:       {len(successful)} ({100*len(successful)/len(results):.1f}%)")
    print(f"Failed:           {len(failed)}")
    
    if failed:
        print("\nFailed runs:")
        for r in failed:
            print(f"  - {r['input_file']}: {r.get('error', 'Unknown error')}")
    
    # Calculate statistics on runtime
    if successful:
        runtimes = [r.get('runtime', 0) for r in successful]
        print(f"\nRuntime statistics (successful runs):")
        print(f"  Min:  {min(runtimes):.2f}s")
        print(f"  Max:  {max(runtimes):.2f}s")
        print(f"  Mean: {np.mean(runtimes):.2f}s")
        print(f"  Total: {sum(runtimes):.2f}s")
    
    return results

# Example usage
# results = run_parameter_sweep_robust(input_files, max_workers=4, timeout=300)

## Example 5: Creating Results Summary Table

In [ ]:
def create_results_summary(scenarios, output_files):
    """
    Parse multiple AERMOD outputs and create a summary table.
    
    Args:
        scenarios: List of scenario names/parameters
        output_files: List of AERMOD .out files
    
    Returns:
        DataFrame with summary statistics
    """
    parser = AERMODOutputParser()
    summary_data = []
    
    for scenario, out_file in zip(scenarios, output_files):
        try:
            results = parser.parse_output_file(out_file)
            
            if results['concentrations']:
                # Extract concentrations from all averaging periods
                for i, conc_df in enumerate(results['concentrations']):
                    max_conc = conc_df['CONC'].max()
                    max_x = conc_df.loc[conc_df['CONC'].idxmax(), 'X']
                    max_y = conc_df.loc[conc_df['CONC'].idxmax(), 'Y']
                    
                    summary_data.append({
                        'scenario': scenario,
                        'avg_period': f"Period_{i+1}",
                        'max_conc': max_conc,
                        'max_x': max_x,
                        'max_y': max_y,
                        'distance': np.sqrt(max_x**2 + max_y**2)
                    })
        except Exception as e:
            print(f"Error processing {out_file}: {e}")
    
    df_summary = pd.DataFrame(summary_data)
    return df_summary

# Example usage:
# scenarios = [f"Height_{h}m" for h in stack_heights]
# output_files = [f.replace('.inp', '.out') for f in input_files]
# summary = create_results_summary(scenarios, output_files)
# print(summary.to_string(index=False))
# summary.to_csv('parameter_sweep_summary.csv', index=False)

## Summary

This notebook demonstrated:

1. **Stack height optimization** - Finding minimum compliant height
2. **Emission rate sensitivity** - Verifying linear scaling
3. **Meteorological comparison** - Year-to-year variability
4. **Batch processing** - Parallel execution with error handling
5. **Results analysis** - Parsing and summarizing multiple outputs

### Key Techniques

- **Parametric studies**: Systematically vary one parameter while holding others constant
- **Batch processing**: Use `AERMODRunner.run_batch()` with parallel workers
- **Results parsing**: Extract and compare maximum concentrations across scenarios
- **Visualization**: Plot trends to identify optimal parameters
- **Error handling**: Robust processing with timeout and failure reporting

### Best Practices

1. **Start small**: Test with 2-3 scenarios before full sweep
2. **Use timeouts**: Prevent hung processes
3. **Parallel processing**: Use num_workers for faster batch runs
4. **Document scenarios**: Clear naming conventions (e.g., `height_sweep_h50m.inp`)
5. **Validate results**: Check for convergence and reasonable values

### Next Steps

- **Notebook 05**: Visualization Techniques
- **Notebook 06**: Volume Source Modeling